# Download Permafrost Data

The [Global Terrestrial Network for Permafrost - Database](http://www.gtnpdatabase.org/) contains a repository on data from permafrost sites. Data for both
 * Thaw Depths (aka Active Layer Thickness)
 * Temperatures
are available.

Unfortunately, the site focuses on making the data available individually involving a lot of user interaction. For analyses at scale, this proves inefficient. This notebook downloads the data locally so you can analyse the data yourself.

We are trying to build a simplified version of the GTN-P data with as few tables as possible. With the source using normalized tables, consistency of GTN-P is generally good, so, for the sake of data science ease of use, we use "fat" tables.

![Data Model](Data_Model.png)


## Packages

There are a few packages which may not be part of a default installation. Also, some may require an installation at the OS level outside the realm of pipenv, conda, or pip.
* [geopandas](https://geopandas.org/) is required to save data with geospatial information
* [PostgreSQL](https://www.postgresql.org/) and its Geospatial Information System extension [PostGIS](https://postgis.net/install/) are required locally. We will try to make a docker package available. Note that PostGIS needs to be enabled per-database. For Mac users, consider [Postgret.app](https://postgresapp.com/), for Windows users, consider [PostgreSQL Portable](https://github.com/garethflowers/postgresql-portable)
* [pycountry](https://pypi.org/project/pycountry/) is used to standardise country names and look up their ISO codes
* [Selenium](https://pypi.org/project/selenium/) is required to load pages that use JavaScript to build their content. This generally requires some installation on the OS as Selenium needs a working web browser to render these pages.
* [tqdm](https://github.com/tqdm/tqdm) for progress bars (although we could also use ipywidgets progress bar, but tqdm also measures and displays time per iteration and often ETA)
* We also use sqlalchemy, pandas, numpy, and scipy. To avoid data conversion difficulties, we store local data files as binary files, preference being parquet, so pyarrow or fastparquet may be required.

In [2]:
import io
import pandas as pd
import geopandas as gpd
import shapely
from bs4 import BeautifulSoup
import requests
import zipfile
from pathlib import Path
import time
from tqdm import tqdm
import numpy as np
import pycountry
import glob
import os
import re
from selenium import webdriver
from sqlalchemy import create_engine
import sqlalchemy
from tqdm import tqdm
import json
import datetime
import socket
import urllib
import sys
from scipy.interpolate import interp1d

## Web Crawling using Selenium

The web page is dynamically generated using JavaScript. This makes access the content slightly more challenging. Selenium is a toolset that is created to allow for automated UI testing, but it is also used to crawl websites. 

Depending on your jupyter setup, selenium may be very tricky, or impossible to install, as it actually requires a functioning web browser to be installed on the machine that hosts the jupyter evironment. This is very often not the case for servers or cloud based installations. Should this be the case:
* Open [http://www.gtnpdatabase.org/activelayers](http://www.gtnpdatabase.org/activelayers) in a browser on your workstation
* Try to File->Save the webpage locally
* There will be a hook for you to copy and paste the file further down

In [3]:
options = webdriver.ChromeOptions() # https://stackoverflow.com/questions/50642308
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome(options=options)

In [5]:
wd.get("http://www.gtnpdatabase.org/activelayers")
print("waiting 10 seconds for the JavaScript to complete",end=".")
time.sleep(10) # wait for javascript to finish
print("...done, next page")
activelayers_page = wd.page_source

wd.get("http://www.gtnpdatabase.org/boreholes")
print("waiting 10 seconds for the JavaScript to complete",end=".")
time.sleep(10) # wait for javascript to finish
print("...finished")
boreholes_temps_page = wd.page_source

waiting 10 seconds for the JavaScript to complete....done, next page
waiting 10 seconds for the JavaScript to complete.

In [7]:
# if you could not get Selenium up and running, open the above pages in a browser and save all the content
# then uncomment the next lines and adjust the file names

# with open("activelayers.html","r") as infile:
#    activelayers_page = infile.read()

# with open("boreholes.html","r") as infile:
#    boreholes_temps_page = infile.read()

Unfortunately, the usual way to parse the table using pandas´ `read_html` results in a missing entry, Station _Da Xi Gou  (Glacier Station)_  is not always added. 

In [8]:
df = pd.read_html(wd.page_source)[0] # little known fact, pandas parses tables quite well, note it returns an array
df.columns = [c.lower() for c in df.columns] 
df[df["gtn-p"]=="CN9"]

,name,site,country,gtn-p,vegetation,permafrost,elevation,depth,data,select


### Helper Function to Log Data Issues

As we walk across the data, we will encounter deficiencies. We will set up a log database so that, if possible, deficiencies can be rectified.

In [9]:
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

@static_vars(log=[])
def log_finding(finding):
    finding["datetime_date"] = "{:%Y-%m-%d %H:%M:%S}".format(datetime.datetime.now())
    finding["machine"] = socket.gethostname()
    log_finding.log.append(finding)
    with open("findings.json","w+t") as logfile:
        json.dump(log_finding.log,logfile)

## Database connection

If you plan to put code of yours onto public platforms such as github or gitlab, never, ever, add credentials to your source code. In fact, never ever, even if you have no such plans. We read the credentials and database server from a file names .env, in this repo, copy .env.example and edit the content so it suits your needs. We need a string in the form 

```
postgresql://<user>:<password>@<hostname>/<database>
```

in that file. Often, this is `postgresql://postgres@localhost/postgres` by default.

In [14]:
with open(".env","rt") as idfile:
    connect_string = idfile.read().strip()
conn = create_engine(connect_string)

## Web Page parsing using BeautifulSoup

The download links use an id of the site in their URL. The id field, however, is not visible on the web page, but is in the hyperlink `href` field of the table. These links are contained in c table cell (`<td>`) with an attribute `data-th="Name"`.

The html code looks like

```
<td data-th="Name"><a class="ng-binding" href="/activelayers/view/195/">Allakaket</a></td>
```

and we are after the 195, which is the last forward-slash separated field of the URL. Also, as we already use BeautifulSoup, we take this approach to populate the data frame.

### Activelayers

In [15]:
soup = BeautifulSoup(io.StringIO(activelayers_page))

entries = []
for tr in soup.findAll("tr"):
    entry = {}
    for td in tr.findAll("td"):
        if td["data-th"] == "Gtn-P":
            entry["code"] = td.text
        else:
            entry[td["data-th"].lower()] = td.text
        if td["data-th"] == "Name":
            entry["subsite_id"] = td.contents[0]["href"].split("/")[-2]
            entry["view_url"] = td.contents[0]["href"]
        elif td["data-th"] == "Site":
            entry["site_id"] = td.contents[0]["href"].split("/")[-2]
            entry["site_url"] = td.contents[0]["href"]
    if len(entry)>0:
        entries.append(entry)
dfActiveLayersCatalogue = pd.DataFrame(entries)
dfActiveLayersCatalogue.elevation = pd.to_numeric(dfActiveLayersCatalogue.elevation)
print("{} Entries loaded".format(len(dfActiveLayersCatalogue)))

253 Entries loaded


Add standard country codes and names using `pycountry`.

In [16]:
dfActiveLayersCatalogue["iso2"] = ""
dfActiveLayersCatalogue["iso3"] = ""
dfActiveLayersCatalogue["country_official_name"] = ""
for i,r in dfActiveLayersCatalogue.iterrows():
    info = pycountry.countries.search_fuzzy(r.country)
    dfActiveLayersCatalogue.at[i,"iso2"] = info[0].alpha_2
    dfActiveLayersCatalogue.at[i,"iso3"] = info[0].alpha_3
    if "country_official_name" in info[0].__dict__.keys():
        dfActiveLayersCatalogue.at[i,"country_official_name"] = info[0].country_official_name
    else:
        dfActiveLayersCatalogue.at[i,"country_official_name"] = info[0].name
dfActiveLayersCatalogue["subsite_type"] = "activelayer"
dfActiveLayersCatalogue.head()

,name,subsite_id,view_url,site,site_id,site_url,country,type,code,vegetation,permafrost,elevation,data,select,iso2,iso3,country_official_name,subsite_type
0,56 Mile,227,/activelayers/view/227/,Franklin Bluff,16,/sites/view/16/,United States,Grid,U31 A,Tundra,Continuous,114.0,No,,US,USA,United States,activelayer
1,Abisko,42,/activelayers/view/42/,Abisko,6,/sites/view/6/,Sweden,Grid,S2,Forest Tundra,Discontinuous,507.0,No,,SE,SWE,Sweden,activelayer
2,Akhmelo Channel,88,/activelayers/view/88/,Cherskii,8,/sites/view/8/,Russia,Grid,R17,Forest Tundra,Continuous,5.0,Yes,,RU,RUS,Russian Federation,activelayer
3,Alazeya River,82,/activelayers/view/82/,Cherskii,8,/sites/view/8/,Russia,Grid,R22,Shrub Tundra,Continuous,60.0,Yes,,RU,RUS,Russian Federation,activelayer
4,Alexandria Fiord,27,/activelayers/view/27/,Alexandria Fiord,329,/sites/view/329/,Canada,Grid,C1,Tundra,Continuous,0.0,Yes,,CA,CAN,Canada,activelayer


### Temperatures

In [17]:
soup = BeautifulSoup(io.StringIO(boreholes_temps_page), 'lxml')

entries = []
for tr in soup.findAll("tr"):
    entry = {}
    for td in tr.findAll("td"):
        if td["data-th"] == "Gtn-P":
            entry["code"] = td.text
        else:
            entry[td["data-th"].lower()] = td.text
        if td["data-th"] == "Name":
            entry["subsite_id"] = td.contents[0]["href"].split("/")[-2]
            entry["view_url"] = td.contents[0]["href"]
        elif td["data-th"] == "Site":
            entry["site_id"] = td.contents[0]["href"].split("/")[-2]
            entry["site_url"] = td.contents[0]["href"]
    if len(entry)>0:
        entries.append(entry)
dfBoreholeTempsCatalogue = pd.DataFrame(entries)
dfBoreholeTempsCatalogue.elevation = pd.to_numeric(dfBoreholeTempsCatalogue.elevation)
print("{} Entries loaded".format(len(dfBoreholeTempsCatalogue)))

1380 Entries loaded


There is a typo in the country names, which we want to log.

In [18]:
dfBoreholeTempsCatalogue["iso2"] = ""
dfBoreholeTempsCatalogue["iso3"] = ""
dfBoreholeTempsCatalogue["country_official_name"] = ""
for i,r in dfBoreholeTempsCatalogue.iterrows():
    if r.country.lower() == "kyrgystan":
        log_finding({"topic":"borehole temperatures",
                    "site_id":r.site_id,
                    "subsite_id":r.subsite_id,
                    "site_url":r.site_url,
                    "site":r.site,
                    "diagnosis":"Country spelling incorrect, '{}' is spelled 'Kyrgyzstan'".format(r.country),
                    "fix":"changed to 'Kyrgyzstan'",
                    "needs_attention":False})
        info = pycountry.countries.search_fuzzy("Kyrgyzstan")
    else:
        info = pycountry.countries.search_fuzzy(r.country)
    dfBoreholeTempsCatalogue.at[i,"iso2"] = info[0].alpha_2
    dfBoreholeTempsCatalogue.at[i,"iso3"] = info[0].alpha_3
    if "country_official_name" in info[0].__dict__.keys():
        dfBoreholeTempsCatalogue.at[i,"country_official_name"] = info[0].country_official_name
    else:
        dfBoreholeTempsCatalogue.at[i,"country_official_name"] = info[0].name
        
dfBoreholeTempsCatalogue["subsite_type"] = "temperatures"
dfBoreholeTempsCatalogue.head()

,name,subsite_id,view_url,site,site_id,site_url,country,code,vegetation,permafrost,elevation,depth,data,select,iso2,iso3,country_official_name,subsite_type
0,0 (Deputatsky),1007,/boreholes/view/1007/,Deputatskiy,331,/sites/view/331/,Russia,RU 118,Shrub Tundra,Continuous,462.0,88.00,No,,RU,RUS,Russian Federation,temperatures
1,01TC1,1744,/boreholes/view/1744/,Yukon,73,/sites/view/73/,Canada,CA 196,Grassland,Continuous,18.0,8.00,No,,CA,CAN,Canada,temperatures
2,01TC2,1745,/boreholes/view/1745/,Yukon,73,/sites/view/73/,Canada,CA 197,Grassland,Continuous,95.0,10.00,No,,CA,CAN,Canada,temperatures
3,03TC1,1743,/boreholes/view/1743/,Yukon,73,/sites/view/73/,Canada,CA 195,Grassland,Discontinuous,3.0,6.00,No,,CA,CAN,Canada,temperatures
4,08 (Deputatsky),1008,/boreholes/view/1008/,Deputatskiy,331,/sites/view/331/,Russia,RU 119,Forest Tundra,Continuous,473.0,96.00,No,,RU,RUS,Russian Federation,temperatures


### Combine Borehole Depth and Temperatures Catalogues into one

There is historic reasons these were created separately. Our aim is to assess all permafrost data, so we want one catalogue.

In [19]:
dfBoreholeTempsCatalogue["type"] = "N/A" # pad a missing field which is only present in the borhole data
dfSites = dfActiveLayersCatalogue.append(dfBoreholeTempsCatalogue).reset_index()
# convert Yes/No text into boolean
dfSites["have_data"] = False
idx = dfSites[dfSites["data"] == "Yes"].index
dfSites.at[idx,"have_data"] = True
del dfSites["data"]
dfSites

,index,name,subsite_id,view_url,site,site_id,site_url,country,type,code,vegetation,permafrost,elevation,select,iso2,iso3,country_official_name,subsite_type,depth,have_data
0,0,56 Mile,227,/activelayers/view/227/,Franklin Bluff,16,/sites/view/16/,United States,Grid,U31 A,Tundra,Continuous,114.0,,US,USA,United States,activelayer,NaN,False
1,1,Abisko,42,/activelayers/view/42/,Abisko,6,/sites/view/6/,Sweden,Grid,S2,Forest Tundra,Discontinuous,507.0,,SE,SWE,Sweden,activelayer,NaN,False
2,2,Akhmelo Channel,88,/activelayers/view/88/,Cherskii,8,/sites/view/8/,Russia,Grid,R17,Forest Tundra,Continuous,5.0,,RU,RUS,Russian Federation,activelayer,NaN,True
3,3,Alazeya River,82,/activelayers/view/82/,Cherskii,8,/sites/view/8/,Russia,Grid,R22,Shrub Tundra,Continuous,60.0,,RU,RUS,Russian Federation,activelayer,NaN,True
4,4,Alexandria Fiord,27,/activelayers/view/27/,Alexandria Fiord,329,/sites/view/329/,Canada,Grid,C1,Tundra,Continuous,0.0,,CA,CAN,Canada,activelayer,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,1375,Zima railst,1677,/boreholes/view/1677/,Irkutsk,369,/sites/view/369/,Russia,N/A,,Other,Isolated patches,456.0,,RU,RUS,Russian Federation,temperatures,3.00,True
1629,1376,Zugspitze 01/07 (W),859,/boreholes/view/859/,Zugspitze,263,/sites/view/263/,Germany,N/A,DE 01,Other,Isolated Patches,2922.0,,DE,DEU,Germany,temperatures,44.00,False
1630,1377,Zugspitze 02/07 (E),858,/boreholes/view/858/,Zugspitze,263,/sites/view/263/,Germany,N/A,DE 02,Other,Isolated Patches,2922.0,,DE,DEU,Germany,temperatures,58.00,False
1631,1378,Zugspitze tunnel,1798,/boreholes/view/1798/,Zugspitze,263,/sites/view/263/,Germany,N/A,,No Vegetation,Mountain,2785.0,,DE,DEU,Germany,temperatures,50.00,False


In [20]:
Path("./sun/catalogue").mkdir(exist_ok=True,parents=True)
dfSites.to_parquet("./sun/catalogue/sites.catalogue.parquet")

In [39]:
#dfSites = pd.read_parquet("./sun/catalogue/sites.catalogue.parquet")

## Download Site Metadata

Each site has one landing page which describes more about its location and some subsites. In particular, we are after Observation Type, Area, Latitude/Longitude, which is not part of the previous dataset.

As all data we want are inside html tables, we use pandas' `read_html`, which returns a list of dataframes.

In [25]:
alldata = []

site_urls = dfSites.site_url.unique()

for site_url in tqdm(site_urls):
    dfGeneral,dfSiteCentre,dfBBox = pd.read_html("http://www.gtnpdatabase.org{}".format(site_url))
    alldata.append({"site_id":site_url.split("/")[-2],
                   "observation_type":dfGeneral.iloc[1].values[1],
                  "area":pd.to_numeric(dfGeneral.iloc[2].values[1].replace(",","").split(" ")[0]),
                  "lon":pd.to_numeric(dfSiteCentre.iloc[0].values[1].replace("°","")),
                  "lat":pd.to_numeric(dfSiteCentre.iloc[1].values[1].replace("°","")),
                  "bbox_lat_max":pd.to_numeric(dfBBox.iloc[0].values[1].replace("°","").split(":")[1]),
                  "bbox_lat_min":pd.to_numeric(dfBBox.iloc[2].values[1].replace("°","").split(":")[1]),
                  "bbox_lon_min":pd.to_numeric(dfBBox.iloc[1].values[0].replace("°","").split(":")[1]),
                  "bbox_lon_max":pd.to_numeric(dfBBox.iloc[1].values[2].replace("°","").split(":")[1])})
    time.sleep(.25)

dfSiteInfo.observation_type = dfSiteInfo.observation_type.replace(np.nan, '', regex=True)
dfSiteInfo = pd.DataFrame(alldata)

100%|██████████| 352/352 [03:14<00:00,  1.81it/s]


In [40]:
dfSiteInfo.observation_type = dfSiteInfo.observation_type.replace(np.nan, '', regex=True)

In [41]:
dfSites = dfSites.merge(dfSiteInfo,on="site_id")
Path("./mercury/catalogue").mkdir(exist_ok=True,parents=True)
dfSites.to_parquet("./mercury/catalogue/sites.catalogue.parquet")

In [42]:
gdfSites = gpd.GeoDataFrame(dfSites, geometry=gpd.points_from_xy(dfSites.lon,dfSites.lat)).set_crs(epsg=4326)
for c in ["subsite_id","site_id","elevation","depth","area","lat","lon","bbox_lat_max","bbox_lat_min","bbox_lon_min","bbox_lon_max"]:
    gdfSites[c] = pd.to_numeric(gdfSites[c])
del gdfSites["index"]
del gdfSites["select"]

In [43]:
gdfSites.to_postgis("t_permafrost_sites",con=conn,dtype={"name":sqlalchemy.types.VARCHAR(100),
                                                                          "subsite_id":sqlalchemy.types.INT,
                                                                          "view_url":sqlalchemy.types.VARCHAR(100),
                                                                          "site":sqlalchemy.types.VARCHAR(100),
                                                                          "site_id":sqlalchemy.types.INT,
                                                                          "site_url":sqlalchemy.types.VARCHAR(100),
                                                                          "country":sqlalchemy.types.VARCHAR(40),
                                                                          "type":sqlalchemy.types.VARCHAR(20),
                                                                          "code":sqlalchemy.types.VARCHAR(20),
                                                                          "vegetation":sqlalchemy.types.VARCHAR(50),
                                                                          "permafrost":sqlalchemy.types.VARCHAR(50),
                                                                          "iso2":sqlalchemy.types.VARCHAR(2),
                                                                          "iso3":sqlalchemy.types.VARCHAR(3),
                                                                          "country_official_name":sqlalchemy.types.VARCHAR(80),
                                                                          "subsite_type":sqlalchemy.types.VARCHAR(20),
                                                                          "observation_type":sqlalchemy.types.VARCHAR(20),
                                                                      },if_exists="replace")

In [46]:
#gdfSites = gpd.read_postgis("SELECT * FROM t_permafrost_sites",con=conn,geom_col="geometry")

## Subsite Metadata

Similar to sites, which are a collection of one or more subsites (which actually resemble the borholes, or borehole arrays), subsites have their own metadata site. Metadata there varies between activelayer and temperatures subsites, also, some tables are not present in the html code if no data are associated with them.